loads processed data, calculates crop domain, saves in the correct location

In [1]:
import sys
import importlib
import os
import logging

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.ndimage.filters as filters
from distributed import LocalCluster, Client
from netCDF4 import Dataset, date2num, num2date

In [2]:
import letkf_forecasting.letkf_forecasting as lf
import letkf_forecasting.prepare_sat_data as prep
import letkf_forecasting.random_functions as rf

In [3]:
sensor_data = pd.read_hdf('/home/travis/python_code/letkf_forecasting_other_things/data/sensor_data.h5')
sensor_CI = pd.read_hdf('/home/travis/python_code/letkf_forecasting_other_things/data/sensor_CI.h5')
sensor_loc = pd.read_hdf('/home/travis/python_code/letkf_forecasting_other_things/data/sensor_loc.h5')

In [4]:
sensor_loc.sort_values(by='id', inplace=True)

In [5]:
year = 2014
month = 5
day = 29
suffix = '_{month}_{day}'.format(month=month, day=day)
file = '/home/travis/python_code/letkf_forecasting_other_things/data/for' + suffix + '/' + '{var}'

In [6]:
csi = pd.read_hdf(file.format(var='csi.h5'))
x_sat = np.load(file.format(var='x.npy'))
y_sat = np.load(file.format(var='y.npy'))
domain_shape = np.load(file.format(var='domain_shape.npy'))
U = pd.read_hdf(file.format(var='U.h5'))
V = pd.read_hdf(file.format(var='V.h5'))
U_shape = np.load(file.format(var='U_shape.npy'))
V_shape = np.load(file.format(var='V_shape.npy'))

U.index = U.index.tz_convert('MST')
V.index = V.index.tz_convert('MST')
csi.index = csi.index.tz_convert('MST')

In [7]:
## filter winds
filter_len = 30*4
U_smooth = U.values.reshape(
    [U.shape[0], U_shape[0], U_shape[1]])
U_smooth = filters.uniform_filter(
    U_smooth, size=(0, filter_len, filter_len))
this_shape = U_smooth.shape
U_smooth = U_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
U_smooth = pd.DataFrame(data=U_smooth, index=U.index)

V_smooth = V.values.reshape(
    [V.shape[0], V_shape[0], V_shape[1]])
V_smooth = filters.uniform_filter(
    V_smooth, size=(0, filter_len, filter_len))
this_shape = V_smooth.shape
V_smooth = V_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
V_smooth = pd.DataFrame(data=V_smooth, index=V.index)

In [8]:
# crop fields

In [9]:
x_range = x_sat
y_range = y_sat
x_sat, y_sat = np.meshgrid(x_sat, y_sat)
x_sat = x_sat.ravel()
y_sat = y_sat.ravel()

In [10]:
lon_sens = sensor_loc['lon'].values
lat_sens = sensor_loc['lat'].values

In [34]:
x_sens, y_sens = prep.sphere_to_lcc(lat_sens, lon_sens)
x_min = np.floor(x_sens.min())
x_max = np.ceil(x_sens.max())
y_min = np.floor(y_sens.min())
y_max = np.ceil(y_sens.max())
west_east_min = np.argmin(abs(x_min - x_range))
west_east_max = np.argmin(abs(x_max - x_range))
south_north_min = np.argmin(abs(y_min - y_range))
south_north_max = np.argmin(abs(y_max - y_range))

In [30]:
x_min

240.0

In [29]:
x_range

array([  80.  ,   80.25,   80.5 , ...,  440.5 ,  440.75,  441.  ])

In [35]:
west_east_min

640

In [33]:
west_east_max

800

In [36]:
south_north_min

600

In [37]:
south_north_max

824

In [12]:
# max_forecast = 45
# max_forecast = 60
max_forecast = 60
if U.values.mean()>0:
    U_max = U.values.max() # know U is positive
    left = int(U_max*60*max_forecast/250) + 10
    right = 10
else:
    U_min = U.values.min() # know U is negative
    left = 10
    right = int(abs(U_min)*60*max_forecast/250) + 10

if V.values.mean()>0:  
    V_max = V.values.max() # know V is positive
    down = int(V_max*60*max_forecast/250) + 10
    up = 10
else:
    V_min = V.values.min()
    down = 10
    up = int(abs(V_min)*60*max_forecast/250) + 10

west_east_range = np.arange(west_east_min - left, west_east_max + right + 1, dtype='int')
west_east_range_stag = np.arange(west_east_min - left, west_east_max + right + 1 + 1, dtype='int')
south_north_range = np.arange(south_north_min - down, south_north_max + up + 1, dtype='int')
south_north_range_stag = np.arange(south_north_min - down, south_north_max + up + 1 + 1, dtype='int')
crop_shape = (south_north_range.size, west_east_range.size)
U_crop_shape = (south_north_range.size, west_east_range_stag.size)
V_crop_shape = (south_north_range_stag.size, west_east_range.size)
U_crop_size = U_crop_shape[0]*U_crop_shape[1]
V_crop_size = V_crop_shape[0]*V_crop_shape[1]
print('X length: ' + 
      str(west_east_range.size))
print('Y length: ' + 
      str(south_north_range.size))

X length: 374
Y length: 654


In [13]:
west_east_crop, south_north_crop = np.meshgrid(west_east_range, south_north_range)
crop_indicies_2d = np.array([south_north_crop.ravel(),
                             west_east_crop.ravel()])
flat_crop_indicies = np.ravel_multi_index(crop_indicies_2d,
                                          domain_shape)

In [14]:
west_east_crop_U, south_north_crop_U = np.meshgrid(west_east_range_stag, south_north_range)
crop_indicies_2d_U = np.array([south_north_crop_U.ravel(),
                               west_east_crop_U.ravel()])
flat_crop_indicies_U = np.ravel_multi_index(crop_indicies_2d_U,
                                            U_shape)

In [15]:
west_east_crop_V, south_north_crop_V = np.meshgrid(west_east_range, south_north_range_stag)
crop_indicies_2d_V = np.array([south_north_crop_V.ravel(),
                               west_east_crop_V.ravel()])
flat_crop_indicies_V = np.ravel_multi_index(crop_indicies_2d_V,
                                            V_shape)

In [16]:
x_crop = x_sat[flat_crop_indicies]
y_crop = y_sat[flat_crop_indicies]

In [17]:
x_crop_range = x_crop.reshape(crop_shape)[0, :]
y_crop_range = y_crop.reshape(crop_shape)[:, 0]

In [18]:
west_east_min = np.argmin(abs(x_min - x_crop_range))
west_east_max = np.argmin(abs(x_max - x_crop_range))
south_north_min = np.argmin(abs(y_min - y_crop_range))
south_north_max = np.argmin(abs(y_max - y_crop_range))
we_error_range = np.arange(west_east_min - 8, west_east_max + 9, dtype='int')
sn_error_range = np.arange(south_north_min - 8, south_north_max + 9, dtype='int')
error_domain_shape = (sn_error_range.size, we_error_range.size)
print('X length: ' + 
      str(we_error_range.size))
print('Y length: ' + 
      str(sn_error_range.size))

X length: 177
Y length: 241


In [19]:
west_east_error_crop, south_north_error_crop = np.meshgrid(
    we_error_range, sn_error_range)
crop_indicies_2d_error = np.array([south_north_error_crop.ravel(),
                             west_east_error_crop.ravel()])
flat_error_domain = np.ravel_multi_index(crop_indicies_2d_error,
                                         crop_shape)

In [20]:
csi_crop = csi[flat_crop_indicies]
csi_crop.columns = np.arange(flat_crop_indicies.size, dtype='int')

In [21]:
U_crop = U_smooth[flat_crop_indicies_U]
U_crop.columns = np.arange(flat_crop_indicies_U.size, dtype='int')
V_crop = V_smooth[flat_crop_indicies_V]
V_crop.columns = np.arange(flat_crop_indicies_V.size, dtype='int')

## saving winds and satellite data

In [22]:
# make file paths

home = os.path.expanduser("~")
file_path = f'{home}/data/{year:04}/{month:02}/{day:02}/'
if not os.path.exists(file_path):
    os.makedirs(file_path)
    
file_path_store = os.path.join(file_path, 'data.nc')
print(file_path_store)

/home2/travis/data/2014/06/12/data.nc


In [22]:
# shape everything back again
x = x_sat.reshape(domain_shape)[0, :]
y = y_sat.reshape(domain_shape)[:, 0]
time = csi.index.tz_convert('UTC').tz_convert(None).to_pydatetime()
time = date2num(time, 'seconds since 1970-1-1')
ci = csi.values.reshape([time.size, domain_shape[0], domain_shape[1]])
time_wind = U.index.tz_convert('UTC').tz_convert(None).to_pydatetime()
time_wind = date2num(time_wind, 'seconds since 1970-1-1')
U_reshape = U.values.reshape(time_wind.size, U_shape[0], U_shape[1])
V_reshape = V.values.reshape(time_wind.size, V_shape[0], V_shape[1])

In [23]:
dx = (x[1] - x[0])
x_stag = np.concatenate([x - dx/2, [x[-1] + dx/2]])
dy = (y[1] - y[0])
y_stag = np.concatenate([y - dy/2, [y[-1] + dy/2]])

In [26]:
x_min

240.0

In [25]:
x[west_east_range]

array([ 237.5 ,  237.75,  238.  ,  238.25,  238.5 ,  238.75,  239.  ,
        239.25,  239.5 ,  239.75,  240.  ,  240.25,  240.5 ,  240.75,
        241.  ,  241.25,  241.5 ,  241.75,  242.  ,  242.25,  242.5 ,
        242.75,  243.  ,  243.25,  243.5 ,  243.75,  244.  ,  244.25,
        244.5 ,  244.75,  245.  ,  245.25,  245.5 ,  245.75,  246.  ,
        246.25,  246.5 ,  246.75,  247.  ,  247.25,  247.5 ,  247.75,
        248.  ,  248.25,  248.5 ,  248.75,  249.  ,  249.25,  249.5 ,
        249.75,  250.  ,  250.25,  250.5 ,  250.75,  251.  ,  251.25,
        251.5 ,  251.75,  252.  ,  252.25,  252.5 ,  252.75,  253.  ,
        253.25,  253.5 ,  253.75,  254.  ,  254.25,  254.5 ,  254.75,
        255.  ,  255.25,  255.5 ,  255.75,  256.  ,  256.25,  256.5 ,
        256.75,  257.  ,  257.25,  257.5 ,  257.75,  258.  ,  258.25,
        258.5 ,  258.75,  259.  ,  259.25,  259.5 ,  259.75,  260.  ,
        260.25,  260.5 ,  260.75,  261.  ,  261.25,  261.5 ,  261.75,
        262.  ,  262

In [25]:
# need more metadata here
with Dataset(file_path_store, 'w') as store:
    store.createDimension('west_east', size=x.size)
    store.createDimension('south_north', size=y.size)
    store.createDimension('we_stag', size=x_stag.size)
    store.createDimension('sn_stag', size=y_stag.size)
    store.createDimension('time', size=time.size)
    store.createDimension('time_wind', size=time_wind.size)
    wenc = store.createVariable('west_east', 'f8', ('west_east',), zlib=True)
    snnc = store.createVariable('south_north', 'f8', ('south_north',), zlib=True)
    we_stagnc = store.createVariable('we_stag', 'f8', ('we_stag',), zlib=True)
    sn_stagnc = store.createVariable('sn_stag', 'f8', ('sn_stag',), zlib=True)
    timenc = store.createVariable('time', 'f8', ('time',), zlib=True)
    time_windnc = store.createVariable('time_wind', 'f8', ('time_wind',), zlib=True)
    cinc = store.createVariable('ci', 'f8', ('time', 'south_north', 'west_east',), zlib=True)
    Unc = store.createVariable('U', 'f8', ('time_wind', 'south_north', 'we_stag',), zlib=True)
    Vnc = store.createVariable('V', 'f8', ('time_wind', 'sn_stag', 'west_east',), zlib=True)
    wenc[:] = x
    snnc[:] = y
    we_stagnc[:] = x_stag
    sn_stagnc[:] = y_stag
    timenc[:] = time
    time_windnc[:] = time_wind
    cinc[:] = ci
    Unc[:] = U_reshape
    Vnc[:] = V_reshape
    timenc.units = 'seconds since 1970-1-1'
    time_windnc.units = 'seconds since 1970-1-1'
    # should move this into forecast_system with calculation
    cinc.we_min_crop = west_east_range[0]
    cinc.we_max_crop = west_east_range[-1]
    cinc.sn_min_crop = south_north_range[0]
    cinc.sn_max_crop = south_north_range[-1]
    Unc.we_min_crop = west_east_range_stag[0]
    Unc.we_max_crop = west_east_range_stag[-1]
    Unc.sn_min_crop = south_north_range[0]
    Unc.sn_max_crop = south_north_range[-1]
    Vnc.we_min_crop = west_east_range[0]
    Vnc.we_max_crop = west_east_range[-1]
    Vnc.sn_min_crop = south_north_range_stag[0]
    Vnc.sn_max_crop = south_north_range_stag[-1]